**Importing required packages**

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p Mydrive
!google-drive-ocamlfuse Mydrive

% cd Mydrive

ls

In [2]:
import numpy as np
X = np.load('X_data.npy')
Y = np.load('Y_data.npy')

In [3]:
IMG_SIZE= 256
NUM_CLASSES = 6

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
K.set_image_data_format('channels_first')


def cnn_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=(3, IMG_SIZE, IMG_SIZE),
                     activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    return model

In [5]:
from keras.optimizers import SGD

model = cnn_model()


In [6]:
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size  =0.1)

In [8]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(279, 3, 256, 256) (31, 3, 256, 256) (279, 6) (31, 6)


In [9]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint


def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

batch_size = 32
epochs = 90

model.fit(X_train, Y_train,batch_size=batch_size,epochs=epochs,callbacks=[LearningRateScheduler(lr_schedule),ModelCheckpoint('model.h5', save_best_only=True)])

Epoch 1/90
279/279 [==============================] - 170s 608ms/step - loss: 1.5813 - acc: 0.3799
Epoch 2/90


/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:435: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


279/279 [==============================] - 169s 606ms/step - loss: 1.5058 - acc: 0.4122
Epoch 3/90
279/279 [==============================] - 169s 605ms/step - loss: 1.4100 - acc: 0.4731
Epoch 4/90
279/279 [==============================] - 169s 605ms/step - loss: 1.3472 - acc: 0.4767
Epoch 5/90
279/279 [==============================] - 170s 608ms/step - loss: 1.1897 - acc: 0.5412
Epoch 6/90
279/279 [==============================] - 169s 604ms/step - loss: 1.2861 - acc: 0.5018
Epoch 7/90
279/279 [==============================] - 169s 606ms/step - loss: 1.3103 - acc: 0.5484
Epoch 8/90
279/279 [==============================] - 169s 605ms/step - loss: 1.0650 - acc: 0.6129
Epoch 9/90
279/279 [==============================] - 165s 592ms/step - loss: 1.1072 - acc: 0.5699
Epoch 10/90
279/279 [==============================] - 164s 588ms/step - loss: 1.1583 - acc: 0.5341
Epoch 11/90
279/279 [==============================] - 164s 589ms/step - loss: 1.0574 - acc: 0.6165
Epoch 12/90
279/279

279/279 [==============================] - 154s 554ms/step - loss: 0.5693 - acc: 0.7742
Epoch 85/90
279/279 [==============================] - 154s 553ms/step - loss: 0.5560 - acc: 0.7957
Epoch 86/90
279/279 [==============================] - 154s 553ms/step - loss: 0.5476 - acc: 0.8029
Epoch 87/90
279/279 [==============================] - 154s 553ms/step - loss: 0.5141 - acc: 0.8208
Epoch 88/90
279/279 [==============================] - 154s 554ms/step - loss: 0.5073 - acc: 0.8029
Epoch 89/90
279/279 [==============================] - 154s 553ms/step - loss: 0.5300 - acc: 0.8172
Epoch 90/90
279/279 [==============================] - 154s 553ms/step - loss: 0.5422 - acc: 0.8065


In [10]:
# predict and evaluate
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
y_pred = model.predict_classes(X_test)
y_pred_encode = y_pred.reshape(len(y_pred),1)
OneHotEncode = OneHotEncoder(sparse= False)
y_pred_OneHotEncoded = OneHotEncode.fit_transform(y_pred_encode)
print("Accuracy: ", accuracy_score(Y_test, y_pred_OneHotEncoded))

Accuracy:  0.9032258064516129


In [53]:
#!pip3 install h5py

In [11]:
from keras.models import load_model
# Load the model
bot_model_filename = 'My_Trained_model.h5'
model.save(bot_model_filename)

In [12]:
model_loaded = load_model('My_Trained_model.h5')

In [18]:
#!pip3 install --user scikit-image

  Using cached https://files.pythonhosted.org/packages/64/18/402284ae09b6a2a8694e707b4924a1a9573bb4aa67923cf487978555e822/scikit_image-0.13.1-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/5a/1f/203c8886c7b94286ddbbf418779f4303774b65c20474a8b554598c483e90/PyWavelets-0.5.2-cp35-cp35m-manylinux1_x86_64.whl


In [19]:
import numpy as np
from skimage import color, exposure, transform

NUM_CLASSES = 6
IMG_SIZE = 256


def preprocess_img(img):
    # Histogram normalization in v channel
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img = color.hsv2rgb(hsv)

    # central square crop
    min_side = min(img.shape[:-1])
    centre = img.shape[0] // 2, img.shape[1] // 2
    img = img[centre[0] - min_side // 2:centre[0] + min_side // 2,
              centre[1] - min_side // 2:centre[1] + min_side // 2,
              :]

    # rescale to standard size
    img = transform.resize(img, (IMG_SIZE, IMG_SIZE))

    # roll color axis to axis 0
    img = np.rollaxis(img, -1)

    return img

In [24]:
from skimage import io
import os
import glob

def get_class(turn_numeric):
    return{
        0:"left",
        1:"right",
        2:"stop",
        3:"u",
        4:"traffic",
        5: "misc"        
    }[turn_numeric]

#img_path = "camera_right.png"
#img_path = "camera_left.png"
img_path = "camera.png"
img = preprocess_img(io.imread(img_path))

X_predict = np.array(img, dtype='float32')
X_predict_new = np.expand_dims(X_predict, axis =0)
turn_num_ = model_loaded.predict_classes(X_predict_new)
turn = get_class(turn_num_[0])

/home/sudhanshu/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [25]:
turn

'stop'